In [1]:
import numpy as np

In [19]:
data = np.loadtxt("tic-tac-toe.data.txt", dtype=str, delimiter=',')

In [20]:
np.random.shuffle(data)

In [21]:
X = data[:, :-1]

In [37]:
y = data[:, -1] == 'positive'

In [23]:
import pandas

In [24]:
X = pandas.DataFrame(X)

In [25]:
X = pandas.get_dummies(X)

In [26]:
X = np.array(X)

In [38]:
split = 800
train_X = X[:split]
train_y = y[:split]
test_X = X[split:]
test_y = y[split:]

In [71]:
def classify_one(pos_samples, neg_samples, sample):
    for pos in pos_samples:
        intersect = pos * sample
        for neg in neg_samples:
            if np.all(intersect <= neg):
                break
        else:
            return True
    return False


def classify(train_X, train_y, test_X, classifier):
    pos_samples = train_X[train_y]
    neg_samples = train_y[~train_y]
    return [classifier(pos_samples, neg_samples, sample) for sample in test_X]

In [44]:
res = classify(train_X, train_y, test_X)

In [45]:
def accuracy(a, b):
    return np.sum(a == b) / len(a)

In [46]:
accuracy(res, test_y)

0.7025316455696202

In [75]:
def crossval(X, y, split_num, classifier):
    splits = np.linspace(0, len(X), split_num + 1, dtype=int)
    X_splits = [X[splits[i]:splits[i + 1]] for i in range(split_num)]
    y_splits = [y[splits[i]:splits[i + 1]] for i in range(split_num)]
    res = []
    for k in range(split_num):
        train_X = np.concatenate([X_splits[i] for i in range(split_num) if i != k])
        train_y = np.concatenate([y_splits[i] for i in range(split_num) if i != k])
        res.append(accuracy(classify(train_X, train_y, X_splits[k], classifier), y_splits[k]))
    return np.mean(res)

In [73]:
crossval(X, y, 10, classify_one)

[  0  95 191 287 383 479 574 670 766 862 958]
[0.6526315789473685, 0.6354166666666666, 0.65625, 0.6458333333333334, 0.6770833333333334, 0.6947368421052632, 0.6666666666666666, 0.5520833333333334, 0.6354166666666666, 0.71875]


0.6534868421052632

In [84]:
def int_size(pos_samples, neg_samples, a, b):
    return np.sum(a * b)

def supp(pos_samples, neg_samples, a, b):
    inter = a * b
    return np.sum([np.all(inter <= pos) for pos in pos_samples]) * 1.0 / len(pos_samples)

def conf(pos_samples, neg_samples, a, b):
    return np.sum([np.all(inter <= neg) for neg in neg_samples]) * 1.0 / len(neg_samples)

In [86]:
for agg in [np.mean, np.amin, np.amax]:
    for rule in [int_size, supp, conf]:
        def classifier(pos_samples, neg_samples, sample):
            p = agg([rule(pos_samples, neg_samples, sample, pos) for pos in pos_samples])
            n = agg([rule(neg_samples, pos_samples, sample, neg) for neg in neg_samples])
            return p > n
        print(agg, rule, crossval(X, y, 10, classifier))

<function mean at 0x10f907c80> <function int_size at 0x11c624d90> 0.6534868421052632
<function mean at 0x10f907c80> <function supp at 0x1a20b75bf8> 0.34651315789473686


NameError: name 'inter' is not defined

In [77]:
import sklearn

In [78]:
import sklearn.tree

In [79]:
t = sklearn.tree.DecisionTreeClassifier()

In [80]:
t.fit(train_X, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [81]:
accuracy(t.predict(test_X), test_y)

0.9620253164556962